In [ ]:
# importing the dependencies
import pandas as pd
import tensorflow as tf

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer

In [ ]:
!pip install pdfquery   # installing denpendency

In [ ]:
# Extracting all the words of the pdf

from pdfquery import PDFQuery

pdf = PDFQuery('Happy Money.pdf')
pdf.load()

# Use CSS-like selectors to locate the elements
text_elements = pdf.pq('LTTextLineHorizontal')

# Extract the text from the elements
text = [t.text for t in text_elements]

print(text)

['Be supported ', 'on your ', 'happy ', 'money ', 'journey! ', 'Creating flow and ', 'happiness with ', 'money is most easily ', 'done with others. ', 'If you would like ', 'support, additional ', 'resources, and to ', 'connect to a ', 'community of people ', 'learning to embrace ', 'happy money, please ', 'join us at ', 'www.KenHonda.com ', '/community. ', 'Thank you for downloading this Simon & Schuster ', 'ebook. ', 'Get a FREE ebook when you join our mailing list. Plus, get updates on new releases, ', 'deals, recommended reads, and more from Simon & Schuster. Click below to sign up and ', 'see terms and conditions. ', '', 'Already a subscriber? Provide your email again so we can register this ebook and send ', 'you more of what you like to read. You will continue to receive exclusive offers in your ', 'inbox. ', 'To all the people in ', 'life who have ', 'my ', 'shared ', 'their ', 'experiences with me ', 'and who showed me ', 'things ', 'the beautiful ', 'can ', 'that money ', 'ac

In [ ]:
# taking all the words and forming a paragraph
df = ''
for word in text:
  df = df + str(word)

In [ ]:
tokenizer = Tokenizer()

In [ ]:
df

'Be supported on your happy money journey! Creating flow and happiness with money is most easily done with others. If you would like support, additional resources, and to connect to a community of people learning to embrace happy money, please join us at www.KenHonda.com /community. Thank you for downloading this Simon & Schuster ebook. Get a FREE ebook when you join our mailing list. Plus, get updates on new releases, deals, recommended reads, and more from Simon & Schuster. Click below to sign up and see terms and conditions. Already a subscriber? Provide your email again so we can register this ebook and send you more of what you like to read. You will continue to receive exclusive offers in your inbox. To all the people in life who have my shared their experiences with me and who showed me things the beautiful can that money achieve, as well as the ugly ways that it this can affect us, book is for you. Is Your Money Smiling? A few years ago, I had a unique experience that became th

In [ ]:
sen = df.split('.')
sen = sen[:501]

In [ ]:
tokenizer.fit_on_texts(sen)

In [ ]:
len(tokenizer.word_index)

1902

In [ ]:
# Extracting the sentences from the paragraph and converting each sentences into vector
ip_sequences = []
for sentence in sen:
  tokenized_sentence = tokenizer.texts_to_sequences([sentence])[0]

  for i in range(1,len(tokenized_sentence)):
    ip_sequences.append(tokenized_sentence[:i+1])

In [ ]:
ip_sequences

[[44, 813],
 [44, 813, 30],
 [44, 813, 30, 31],
 [44, 813, 30, 31, 59],
 [44, 813, 30, 31, 59, 4],
 [44, 813, 30, 31, 59, 4, 814],
 [44, 813, 30, 31, 59, 4, 814, 531],
 [44, 813, 30, 31, 59, 4, 814, 531, 129],
 [44, 813, 30, 31, 59, 4, 814, 531, 129, 2],
 [44, 813, 30, 31, 59, 4, 814, 531, 129, 2, 178],
 [44, 813, 30, 31, 59, 4, 814, 531, 129, 2, 178, 15],
 [44, 813, 30, 31, 59, 4, 814, 531, 129, 2, 178, 15, 4],
 [44, 813, 30, 31, 59, 4, 814, 531, 129, 2, 178, 15, 4, 10],
 [44, 813, 30, 31, 59, 4, 814, 531, 129, 2, 178, 15, 4, 10, 60],
 [44, 813, 30, 31, 59, 4, 814, 531, 129, 2, 178, 15, 4, 10, 60, 532],
 [44, 813, 30, 31, 59, 4, 814, 531, 129, 2, 178, 15, 4, 10, 60, 532, 384],
 [44, 813, 30, 31, 59, 4, 814, 531, 129, 2, 178, 15, 4, 10, 60, 532, 384, 15],
 [44,
  813,
  30,
  31,
  59,
  4,
  814,
  531,
  129,
  2,
  178,
  15,
  4,
  10,
  60,
  532,
  384,
  15,
  65],
 [25, 13],
 [25, 13, 70],
 [25, 13, 70, 54],
 [25, 13, 70, 54, 815],
 [25, 13, 70, 54, 815, 816],
 [25, 13, 70, 54,

In [ ]:
# finding the max length of the input sequence
max_len = max([len(x) for x in ip_sequences])

In [ ]:
max_len

111

In [ ]:
# applying pre zero padding to the sequences
from tensorflow.keras.preprocessing.sequence import pad_sequences
padded_sequences = pad_sequences(ip_sequences,maxlen = max_len,padding = 'pre')

In [ ]:
x = padded_sequences[:,:-1] # input sequences

In [ ]:
y = padded_sequences[:,-1] # output column

In [ ]:
 x.shape,y.shape

((9040, 110), (9040,))

In [ ]:
from tensorflow.keras.utils import to_categorical
y = to_categorical(y,num_classes = 1903)     # one hot encoding

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

In [ ]:
model = Sequential()
model.add(Embedding(1903,100,input_length = 110))
model.add(LSTM(150))
model.add(Dense(1903,activation = 'softmax'))

In [ ]:
model.compile(loss = 'categorical_crossentropy',optimizer = 'adam',metrics = ['accuracy'])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 110, 100)          190300    
                                                                 
 lstm (LSTM)                 (None, 150)               150600    
                                                                 
 dense (Dense)               (None, 1903)              287353    
                                                                 
Total params: 628253 (2.40 MB)
Trainable params: 628253 (2.40 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
model.fit(x,y,epochs = 100)

Epoch 1/100
283/283 [==============================] - 32s 86ms/step - loss: 6.4774 - accuracy: 0.0287
Epoch 2/100
283/283 [==============================] - 8s 27ms/step - loss: 6.0707 - accuracy: 0.0400
Epoch 3/100
283/283 [==============================] - 5s 16ms/step - loss: 5.9240 - accuracy: 0.0503
Epoch 4/100
283/283 [==============================] - 4s 14ms/step - loss: 5.7572 - accuracy: 0.0617
Epoch 5/100
283/283 [==============================] - 4s 13ms/step - loss: 5.5322 - accuracy: 0.0835
Epoch 6/100
283/283 [==============================] - 3s 12ms/step - loss: 5.2685 - accuracy: 0.1039
Epoch 7/100
283/283 [==============================] - 3s 12ms/step - loss: 5.0011 - accuracy: 0.1223
Epoch 8/100
283/283 [==============================] - 3s 11ms/step - loss: 4.7447 - accuracy: 0.1378
Epoch 9/100
283/283 [==============================] - 3s 11ms/step - loss: 4.4960 - accuracy: 0.1558
Epoch 10/100
283/283 [==============================] - 3s 10ms/step - loss: 4.25

In [ ]:
text = 'Hello my' # Be supported on your happy money journey
for i in range(10):

  # tokenize
  token_text = tokenizer.texts_to_sequences([text])[0]
  # padding
  padding_input = pad_sequences([token_text],maxlen = 110,padding = 'pre')
  # predict
  pred = np.argmax(model.predict(padding_input))

  for word,index in tokenizer.word_index.items():
    if index == pred:
      text = text + ' ' + word
print(text)

1/1 [==============================] - 0s 21ms/step
Hello my father was an accountant with a successful private business and


In [ ]:
import numpy as np